**MySQL别名**

有时候列的名称是一个表达式，使得查询的输出很难理解。此时可以给出一个描述性名称，这个名称称作别名。    
查询选择员工的名字、姓氏，并将其组合起来生成别名。如果别名有空格则需要使用''括起来。        
**SELECT CONCAT_WS(' ',firstName,lastName) AS 'Full Name' FROM employees LIMIT 10;**

可以使用ORDER BY, GROUP BY和HAVING子句中的列别名来引用该列。查询使用ORDER BY子句中的别名按字母顺序排列员工的名字。    
**SELECT CONCAT_WS(' ',firstName,lastName) AS \`Full Name\` FROM employees ORDER BY \`Full Name\` LIMIT 10;**    
特别注意:Full Name两边的符号是反引号\`，并不是单引号'，如果使用单引号，起别名可以，但是ORDER BY的时候会不起作用。    

查询总金额大于60000的订单，它在GROUP BY 和HAVING子句中使用别名。    
**SELECT orderNumber \`Order no.\`,SUM(quantityOrdered\*priceEach) total FROM orderdetails GROUP BY \`Order no.\` HAVING total > 60000;**

注意：不能在WHERE子句中使用列别名。原因是当MySQL评估求值WHERE子句时，SELECT子句中指定的列值可能尚未确定。

**MySQL表别名**

可以使用别名为表添加不同的名称。使用AS关键字给表名称定义别名。像列名一样，AS关键字是可选的，一般在包含INNER JOIN,LEFT JOIN,SELF JOIN子句和子查询的语句中使用表别名。

示例：客户(customers)和订单(orders)表，它们的ER图如下所示。![ER-customers-orders](./images/ER-customers-orders.png)

两个表都具有相同的列名称：customerNumber。如果不使用表别名来指定是哪个表中的customerNumber列，则执行查询时将收到类似以下错误消息： 

Error Code: 1052. Column 'customerNumber' in on clause is ambiguous

为了避免此错误，应该使用表别名来限定customerNumber列：    
**SELECT customerName,COUNT(o.orderNumber) total FROM customers c INNER JOIN orders o ON c.customerNumber = o.customerNumber GROUP BY customerName HAVING total >= 5 ORDER BY total DESC;**

上面的查询从客户(customers)和订单(orders)表中选择客户名称和订单数量。它使用c作为customers表别名，o作为orders表的别名。customers和orders表中的列通过表别名(c 和 o)引用。

如果不使用表别名，则必须使用表名称来引用其列，这样会使得查询冗长且可读性较低。    
**SELECT customerName,COUNT(orderNumber) total FROM customers INNER JOIN orders ON customers.customerNumber = orders.customerNumber GROUP BY customerName HAVING total >= 5 ORDER BY total DESC;**

**MySQL INNER JOIN 子句**

INNER JOIN 子句将一个表中的行与其他表中的行进行匹配，并允许从两个表中查询包含列的行记录。INNER JOIN 子句是SELECT语句的可选部分，它出现在FROM子句之后。在使用INNER JOIN子句之前，必须指定以下条件：
- 首先，在FROM子句中指定主表。
- 其次，表中要连接的主表应该出现在INNER JOIN子句中。理论上说，可以连接多个其他表。 但是，为了获得更好的性能，应该限制要连接的表的数量(最好不要超过三个表)。
- 第三，连接条件或连接谓词。连接条件出现在INNER JOIN子句的ON关键字之后。连接条件是将主表中的行与其他表中的行进行匹配的规则。

数据库answer中的产品(products)和产品线(productlines)表做示例，ER图如下![ER-products-lines](./images/ER-products-lines.png)

如果想获取以下数据:    
- 获取products表中的productCode和productName列的值。
- 获取productlines表产品线的描述 - textDescription列的值。   

**SELECT productCode,productName,textDescription FROM products p INNER JOIN productlines pl ON p.productline = pl.productline ORDER BY productCode LIMIT 10;**   
由于两个表的连接是使用相同一个列：productline，因此可以使用以下语法，不需要使用表别名：  
**SELECT productCode,productName,textDescription FROM products INNER JOIN productlines USING(productline) ORDER BY productCode LIMIT 10;**

**MySQL INNER JOIN GROUP BY子句**

以下订单和订单详细表，orders表和orderdetails表的结构如下所示:![ER-orders-orderdetails](./images/ER-orders-orderdetails.png)

可以使用具有GROUP BY子句的INNER JOIN子句从orders和orderdetails表中获取订单号，订单状态和总销售额，如下所示：    
**SELECT o.orderNumber,status,SUM(quantityOrdered\*priceEach) total FROM orders o INNER JOIN orderdetails od ON o.orderNumber = od.orderNumber GROUP BY o.orderNumber ORDER BY total DESC LIMIT 10;**    
亦可以省去别名   
**SELECT orderNumber,status,SUM(quantityOrdered\*priceEach) total FROM orders INNER JOIN orderdetails USING(orderNumber) GROUP BY orderNumber ORDER BY total DESC LIMIT 10;**

**MySQL INNER JOIN使用等于以外的运算符**

使用大于(>)，小于(<)和不等于(<>)运算符的其他运算符来形成连接谓词。
以下查询使用少于(<)连接,来查找代码为S10_1678的产品的销售价格低于制造商建议零售价(MSRP)的所有产品。    
**SELECT orderNumber,productName,msrp,priceEach FROM products p INNER JOIN orderdetails o ON p.productCode = o.productCode AND p.msrp > o.priceEach WHERE p.productCode = 'S10_1678';**

**MySQL LEFT JOIN 子句**

LEFT JOIN子句是SELECT 语句的可选部分，出现在FROM子句之后。LEFT JOIN子句允许从匹配的右表中查询选择行记录，连接左表中的所有行，即使在右表中找不到匹配的行也显示出来，但使用NULL值代替。

使用customers和orders两张表来示例，ER图如下：![ER-customers-orders](./images/ER-customers-orders.png)

在上面的图中：
- 订单(orders)表中的每个订单必须属于客户(customers)表中的客户
- 客户(customers)表中的每个客户在订单(orders)表中可以有零个或多个订单   

要查询每个客户的所有订单，可以使用LEFT JOIN子句，如下所示：    
**SELECT c.customerNumber,c.customerName,o.orderNumber FROM customers c LEFT JOIN orders o ON c.customerNumber = o.customerNumber LIMIT 100,20;**        
要查询每个客户的订单总数，可以使用LEFT JOIN子句，如下所示：    
**SELECT c.customerNumber,c.customerName,COUNT(o.orderNumber) orderCount FROM customers c LEFT JOIN orders o ON c.customerNumber = o.customerNumber GROUP BY customerNumber ORDER BY orderCount LIMIT 20,10;**    
也可以使用USING(customerNumber)来替代ON c.customerNumber = o.customerNumber

**使用LEFT JOIN 子句来查找不匹配的行**

当您想要找到右表中与不匹配的左表中的行时，LEFT JOIN子句非常有用。要查询两个表之间的不匹配行，可以向SELECT语句添加一个WHERE子句，以仅查询右表中的列值包含NULL值的行。要查找没有下过订单的所有客户，请使用以下查询：    
**SELECT customerNumber,customerName,orderNumber FROM customers LEFT JOIN orders USING(customerNumber) WHERE orderNumber IS NULL;**     

**WHERE子句与ON子句中的条件**